<a href="https://colab.research.google.com/github/scskalicky/creative_naming_lsa/blob/main/creative_names_cogsci.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# manually download the most up-to-date English model (lg)
# 
# MUST RESTART RUNTIME AFTER DOING THIS.
# will also update spacy to 3.x
#!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-3.2.0/en_core_web_lg-3.2.0.tar.gz

In [1]:
import pandas as pd
import spacy
nlp = spacy.load('en_core_web_lg')

In [ ]:
# real shit here

# https://github.com/MartinoMensio/spacy-universal-sentence-encoder

# https://stackoverflow.com/questions/52113939/spacy-strange-similarity-between-two-sentences

!pip install git+https://github.com/MartinoMensio/spacy-universal-sentence-encoder.git


In [ ]:
!pip install https://github.com/MartinoMensio/spacy-universal-sentence-encoder/releases/download/v0.4.3/xx_use_lg-0.4.3.tar.gz#xx_use_lg-0.4.3

In [ ]:
import spacy_universal_sentence_encoder
nlp = spacy_universal_sentence_encoder.load_model('en_use_lg')

In [69]:
import spacy
nlp = spacy.load('en_core_web_lg')
nlp.add_pipe('universal_sentence_encoder')

In [71]:
a = 'air ball'
b = 'going to the moon bag'

nlp(a).similarity(nlp(b))

0.26939963742482953

In [78]:
dat = pd.read_csv('/content/drive/MyDrive/creative_naming/creative_naming_raw_answers.csv')
dat = dat[dat['on.task'] == 1]
dat.columns

Index(['subject', 'group', 'item', 'original_answer', 'cleaned_answer',
       'on.task'],
      dtype='object')

In [94]:
nes_dat = dat[dat['group'] == 'NES']
rom_dat = dat[dat['group'] == 'ROM']

0      balloon
1          box
2       carrot
3         fork
4       shovel
        ...   
589        box
590     carrot
591      couch
592    glasses
593     shovel
Name: item, Length: 577, dtype: object

In [171]:
nes_answers = nes_dat[['cleaned_answer', 'item']]
rom_answers = rom_dat[['cleaned_answer', 'item']]

In [86]:
items = list(dat['item'].unique())
items

['balloon',
 'box',
 'carrot',
 'fork',
 'shovel',
 'toilet',
 'couch',
 'glasses',
 'wallet']

In [177]:
nes_answers[nes_answers['item'] == 'balloon']['cleaned_answer'].unique()

array(['air ball', 'rubber helium holder', 'poke popper', 'party bubble',
       'air holder', 'party floater', 'helium globe', 'air toy',
       'party popper', 'toy sac air holder', 'floating ball',
       'gas capsule', 'pop and scare', 'air bomb', 'rubber floater',
       'air floater', 'floating object', 'big plastic colored object',
       'loud popper', 'floating sphere', 'air pack',
       'green air container', 'floating ball of air', 'air container',
       'poppable air container', 'air filler', 'green bubble',
       'green rubber float', 'colorful air holder', 'blown air',
       'inflated decoration', 'blimp bird', 'inflatable rubber',
       'air bag', 'inflated elation', 'decorative rubber float',
       'air filled rubber', 'inflatable bauble', 'chocolate shape maker',
       'breath capturer', 'green helium voice changer', 'rubber blow up',
       'floating air', 'bag of air', 'poppable', 'air trapper',
       'elastic air holder'], dtype=object)

In [186]:
import time
from collections import defaultdict

def sim_dict(answers, items):
  
  output_dict = defaultdict(dict)
  for item in items:
    item_answers = list(answers[answers['item'] == item]['cleaned_answer'])
    print(f'computing similarity for items in {item}')
    #print(item_answers)
    for index, answer in enumerate(item_answers):
      answer_sims_dict = dict()
      print(f'computing similarity for {answer}...')
      print(f'{index+1} of {len(item_answers)}')
      answer_sims = [(x, nlp(answer).similarity(nlp(x))) for x in item_answers]
      output_dict[item][answer] = dict(answer_sims)
      #print(len(output_dict[item].keys()))

  return output_dict

In [ ]:
nes_sims = sim_dict(nes_answers, items)

In [191]:
len(list(nes_sims['balloon'].keys()))

47

In [194]:
import pickle
afile = open(r'/content/drive/MyDrive/creative_naming/nes_sims.pkl', 'wb')
pickle.dump(nes_sims, afile)
afile.close()

In [ ]:
rom_sims = sim_dict(rom_answers,items)

In [ ]:
rom_sims['balloon']

In [196]:
bfile = open(r'/content/drive/MyDrive/creative_naming/rom_sims.pkl', 'wb')
pickle.dump(rom_sims, bfile)
bfile.close()

The goal of this notebook is to see whether or not we can use wordnet to determine the relative heirarchy of categories in the participants answers.


What are the common linguistic properties of high frequency answers?

- repeated answers
- similar answers
- unique answers

What does this mean for creativity? 


So we need a way to group similar answers based on

- the words used
  - ideally we would want to categorize things
- their semantic similarity to one another
- their syntax
- orthographic overlap? 
